In [0]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    BinaryType,
)
from datetime import datetime
import time
import requests

from src.data.data_utils import write_delta_table

spark = SparkSession.builder.getOrCreate()

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "endepunkt_bronze"

# Liste over kommune-IDer du ønsker å hente data for
kommune_ider = ["4621", "4618", "3411", "3422"]

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {bronze_table} (
    nodeid INTEGER,
    geometry BINARY,
    kommune_id STRING,
    ingest_time TIMESTAMP
) USING DELTA
"""
spark.sql(q)

In [0]:
def hent_wkt_koordinater(kommune_id: str, srid="UTM33", max_retries=10):
    """
    Henter WKT-geometri og koordinater for alle noder fra NVDB-apiet.
    """
    node_url = f"https://nvdbapiles.atlas.vegvesen.no/vegnett/api/v4/noder"
    veglenkesekvens_url = (
        f"https://nvdbapiles.atlas.vegvesen.no/vegnett/api/v4/veglenkesekvenser"
    )
    headers = {
        "Accept": "application/json",
        "X-Client": "Systemet for vegobjekter",
    }
    node_params = {"srid": srid, "kommune": kommune_id}
    veglenkesekvens_params = {
        "srid": srid,
        "kommune": kommune_id,
        "vegsystemreferanse": "S",
    }

    schema = StructType(
        [
            StructField("nodeid", StringType(), True),
            StructField("wkt", StringType(), True),
        ]
    )

    new_df = spark.createDataFrame([], schema)

    nodes = []
    attempt = 0
    while True:
        try:
            response = requests.get(
                veglenkesekvens_url,
                headers=headers,
                params=veglenkesekvens_params,
                timeout=10,
            )
            time.sleep(0.5)
            data = response.json()
            objects = data.get("objekter", [])
            for obj in objects:
                porter = obj.get("porter", [])
                for p in porter:
                    nodes.append(p.get("tilkobling", {}).get("nodeid"))
            next_start = data.get("metadata", {}).get("neste", {}).get("start")
            if not next_start:
                break
            veglenkesekvens_params["start"] = next_start

        except requests.exceptions.RequestException as e:
            wait = 2**attempt
            print(
                f"[{kommune_id}] ⚠️ Feil ved henting ({e}), prøver igjen om {wait}s..."
            )
            time.sleep(wait)
            attempt += 1
            if attempt >= max_retries:
                raise e
            continue

    attempt = 0
    while True:
        try:
            response = requests.get(
                node_url, headers=headers, params=node_params, timeout=10
            )
            time.sleep(0.5)
            data = response.json()
            objects = data.get("objekter", [])

            for obj in objects:
                if obj["id"] in nodes:
                    porter = obj.get("porter", [])
                    if len(porter) == 1:
                        portnummer = porter[0].get("tilkobling", {}).get("portnummer")
                        real = portnummer in (1, 2)
                    else:
                        real = False

                    if real:
                        wkt = obj.get("geometri", {}).get("wkt")
                        row = [(obj["id"], wkt)]
                        new_df = new_df.union(
                            spark.createDataFrame(row, ["nodeid", "wkt"])
                        )

            next_start = data.get("metadata", {}).get("neste", {}).get("start")
            if not next_start:
                break
            node_params["start"] = next_start

        except requests.exceptions.RequestException as e:
            wait = 2**attempt
            print(
                f"[{kommune_id}] ⚠️ Feil ved henting ({e}), prøver igjen om {wait}s..."
            )
            time.sleep(wait)
            attempt += 1
            if attempt >= max_retries:
                raise e
            continue

    new_df = new_df.withColumn(
        "geometry", expr("ST_AsBinary(ST_Force2D(ST_GeomFromText(wkt)))")
    )
    new_df = new_df.drop("wkt")
    return new_df

In [0]:
schema = StructType(
    [
        StructField("nodeid", StringType(), True),
        StructField("geometry", BinaryType(), True),
    ]
)

df = spark.createDataFrame([], schema)

for kommune_id in kommune_ider:
    print(f"📡 Henter data for kommune {kommune_id}")
    df = df.union(hent_wkt_koordinater(kommune_id))
df = df.withColumn("kommune_id", lit(kommune_id))
df = df.withColumn("ingest_time", lit(datetime.now()))
write_delta_table(df, bronze_table, id_col="nodeid")